In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.vectorstores.deeplake import DeepLake
from langchain.vectorstores.deeplake import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_cohere import ChatCohere
from langchain_cohere import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.vectorstores import Chroma
import pandas as pd
import os
from llama_index.postprocessor.colbert_rerank import ColbertRerank
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)
import os
from pathlib import Path
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from langchain_community.embeddings import OllamaEmbeddings
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever
from groq import Groq
from llama_index.llms.openai import OpenAI

from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank
from llama_index.llms.ollama import Ollama
import pandas as pd
from IPython.display import display, HTML
from llama_index.embeddings.jinaai import JinaEmbedding
import requests
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain, StuffDocumentsChain
from llama_index.postprocessor.jinaai_rerank import JinaRerank
from llama_index.core.postprocessor import LongContextReorder
import os
from pathlib import Path
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.callbacks import get_openai_callback

from langchain.vectorstores import Chroma
from groq import Groq


from transformers import AutoModel
from numpy.linalg import norm



In [ ]:
#queries = ["What are the steps required to set up a boilerplate project using the Emumba Plugin?"]
queries = [
       "What are the steps required to set up a boilerplate project using the Emumba Plugin?",
        "Can you list some of the key features provided by the Emumba Plugin for React applications?",
        "What is the purpose of the generateFiles function in the project setup generator, and how does it use the options provided?",
        "Describe the role of addDependenciesToPackageJson in the project setup process.",
        "How does the ProjectSetupGeneratorSchema interface influence the behavior of the project setup generator?",
        "Explain how the project configuration is added to the workspace using addProjectConfiguration in the context of the setup process.",
        "Describe the process and the purpose of creating a test project in the beforeAll setup of the emumba-plugin tests.",
        "How does the test for emumba-plugin ensure that the plugin is properly installed and functional within a generated project?",
        "there's a function used to create a test project directory. Output the code snippet that showcases how this directory is created and initialized.",
        '''Given the following incomplete snippet, complete the function to add a specific dependency to the project's package.json. 
            Assume the function addDependenciesToPackageJson is already imported.
            function enhancePackageJson(tree: Tree, projectName: string) {
            // Add 'react-redux' as a dependency
            addDependenciesToPackageJson(tree, projectName, {
                'react-redux': '^7.2.0'
            }, {});
            // Complete the function to also add 'redux' as a dependency
        }'''
]

In [ ]:
groq_api_key="your-key"
os.environ["OPENAI_API_KEY"]  = "your-key"
os.environ["COHERE_API_KEY"] = "your-key"
llm = ChatGroq(groq_api_key=groq_api_key,model_name='llama3-70b-8192')    
client = Groq(api_key="your-key")

### Colbert Reranker

In [ ]:
documents = SimpleDirectoryReader('data/Boilerplate').load_data()
index = VectorStoreIndex.from_documents(documents=documents)

In [ ]:

colbert_reranker = ColbertRerank(
    top_n=4,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)
query_engine = index.as_query_engine(
    similarity_top_k=6,
    node_postprocessors=[colbert_reranker],
)

In [ ]:

def colbertreranker(queries,query_engine):
    result=[]
    file_path = 'positive_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(queries, start=0): 
        ground_truth = df.iloc[i]['ground truth']  
        response = query_engine.query(query)
        context=[]
        for node in response.source_nodes:
            context.append(node.node.get_content())
        result.append({'question': query, 'answer': str(response), 'ground_truths': ground_truth,'contexts':context})
    return result

In [ ]:
result=colbertreranker(queries,query_engine)

### Cohere Reranker

In [ ]:

class Document:
    def __init__(self, content):
        self.page_content = content
        self.metadata = {} 

def read_files(directory_path):
    documents = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_path = Path(root) / file
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                    documents.append(Document(content)) 
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
    return documents

repo_path = 'data/Boilerplate'
documents = read_files(repo_path)
docs_texts = [d.page_content for d in documents]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
docs = text_splitter.split_documents(documents)

In [ ]:
embedding = CohereEmbeddings()
docstore = DeepLake(
    dataset_path="deeplake_vstore",
    embedding=embedding,
    verbose=False,
    num_workers=4,
)
_ = docstore.add_documents(documents=docs)

In [ ]:

cohere_rerank = CohereRerank(
    cohere_api_key="your-key",
    model="rerank-english-v3.0",
)

docstore = DeepLake(
    dataset_path="deeplake_vstore",
    embedding=embedding,
    verbose=False,
    read_only=True,
    num_workers=4,
)

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_rerank,
    
    base_retriever=docstore.as_retriever(
        search_type="similarity",
        search_kwargs={
            "fetch_k": 4,
            "k": 6,
        },
    ),
)

In [ ]:

def cohere_chain(llm,compression_retriever,queries):
    prompt_template = """You are an intelligent chatbot that can answer user's 
    queries. You will be provided with Relevant context based on the user's queries. 
    Your task is to analyze user's query and generate response for the query 
    utiliing the context. 
    
    NEVER generate response to queries for which there is no or irrelevant context.
    
    Context: {context}
    
    Question: {question}
    
    Answer:
    """
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )
    chain_type_kwargs = {"prompt": PROMPT, "verbose": False}
    qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    verbose=False,
    chain_type_kwargs=chain_type_kwargs,
)
    def process_context(source_documents):
        contexts = []
        print(source_documents)
        for doc in source_documents:
            if hasattr(doc, 'page_content'):
                contexts.append(doc.page_content)
            else:
                contexts.append("Invalid document format")
        return contexts
    result=[]
    file_path = 'positive_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(queries, start=0): 
            response = qa.invoke({"query": query})
            
            ground_truth = df.iloc[i]['ground truth']  
            contexts = process_context(response['source_documents'])
            result.append({'question': query, 'answer': response['result'], 'ground_truths': ground_truth,'contexts': contexts})
           

    return result

In [ ]:

result=cohere_chain(llm,compression_retriever,queries)

### RAG FUSION

In [ ]:
documents = SimpleDirectoryReader('data/Boilerplate').load_data()
llm = ChatGroq(groq_api_key=groq_api_key,model_name='llama3-70b-8192')    
embedding=OllamaEmbeddings(model='mxbai-embed-large')
splitter = SentenceSplitter(chunk_size=256)
index = VectorStoreIndex.from_documents(documents, transformations=[splitter])

In [ ]:
QUERY_GEN_PROMPT = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)
vector_retriever = index.as_retriever(similarity_top_k=6)
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=4
)
retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=4,
    num_queries=1,
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=QUERY_GEN_PROMPT
)

In [ ]:
def ragfusion(client,retriever,queries):
    result=[]
    file_path = 'positive_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(queries, start=0): 
        
            nodes_with_scores = retriever.retrieve(query)
            context=[]
            for node in nodes_with_scores:
                print(node)
                context.append(node.text)
            prompt= f"""Answer the question based only on the following context:{context}Question: {query}"""
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model='llama3-70b-8192',
            )
            response=(chat_completion.choices[0].message.content)
            ground_truth = df.iloc[i]['ground truth']  
            result.append({'question': query, 'answer': response, 'ground_truths': ground_truth,'contexts': context})

    return result

In [ ]:
result=ragfusion(client,retriever,queries)

### RANKGPT

In [ ]:
documents = SimpleDirectoryReader('data/Boilerplate').load_data()
index = VectorStoreIndex.from_documents(
    documents,
)
llm = Ollama(model="llama3", request_timeout=1200.0)


In [ ]:
def get_retrieved_nodes(
    query_str, vector_top_k=6, reranker_top_n=4, with_reranker=True
):
    query_bundle = QueryBundle(query_str)
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        print("hi")
        reranker = RankGPTRerank(
            llm=OpenAI(
                model="gpt-3.5-turbo-16k",
                temperature=0.0,
                api_key="your-key",
            ),
            top_n=reranker_top_n,
            verbose=True,
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes
def rankthroughgpt(client,queries):
    result=[]
    file_path = 'positive_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(queries, start=0): 
        
            new_nodes = get_retrieved_nodes(
                query,
                vector_top_k=6,
                reranker_top_n=4,
                with_reranker=True,
            )
            context=[]
            for node in new_nodes:
                print(new_nodes)
                context.append(node.text)
                
            prompt= f"""Answer the question based only on the following context:{context}Question: {query}"""
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model='llama3-70b-8192',
            )
            print("kk")
            response=(chat_completion.choices[0].message.content)
            ground_truth = df.iloc[i]['ground truth']  
            result.append({'question': query, 'answer': response, 'ground_truths': ground_truth,'contexts': context})

    return result

In [ ]:
result=rankthroughgpt(client,queries)

### Jina AI reranker

In [ ]:
documents = SimpleDirectoryReader('data/Boilerplate').load_data()
splitter = SentenceSplitter(chunk_size=256)
index = VectorStoreIndex.from_documents(documents, transformations=[splitter])

In [ ]:
jina_api_key = "your-key"
jina_embeddings = JinaEmbedding(api_key=jina_api_key)
jina_rerank = JinaRerank(api_key=jina_api_key, top_n=4)
query_engine = index.as_query_engine(
    similarity_top_k=6, node_postprocessors=[jina_rerank]
)

In [ ]:
def jinarerank(client,query_engine,queries):
    result=[]
    file_path = 'positive_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(queries, start=0): 
            response = query_engine.query(query)
            context=[]
            scores=[]
            for node in response.source_nodes:
                text = node.text
                context.append(node.text)
                scores.append(node.score)
            prompt= f"""Answer the question based only on the following context:{context}Question: {query}"""
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model='llama3-70b-8192',
            )
            response=(chat_completion.choices[0].message.content)
            ground_truth = df.iloc[i]['ground truth']  
            result.append({'question': query, 'answer': response, 'ground_truths': ground_truth,'contexts': context})

    return result,scores

In [ ]:
result,scores=jinarerank(client,query_engine,queries)

### Long context reorder

In [ ]:
class Document:
    def __init__(self, content):
        self.page_content = content
        self.metadata = {} 

def read_files(directory_path):
    documents = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_path = Path(root) / file
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                    documents.append(Document(content)) 
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
    return documents

repo_path = 'data/Boilerplate'
documents = read_files(repo_path)
docs_texts = [d.page_content for d in documents]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
docs = text_splitter.split_documents(documents)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
retriever = Chroma.from_texts(docs_texts, embedding=embeddings).as_retriever(
    search_kwargs={"k": 2}
)

In [ ]:
def process_context(contexts,source_documents):
        for doc in source_documents:
            if hasattr(doc, 'page_content'):
                contexts.append(doc.page_content)
            else:
                contexts.append("Invalid document format")
        return contexts

In [ ]:
def longcontextreorder(retriever,queries):
    result=[]
    file_path = 'positive_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(queries, start=0): 
        docs = retriever.invoke(query)
        reordering = LongContextReorder()
        reordered_docs = reordering.transform_documents(docs)
        contexts=[]
        contexts = process_context(contexts,reordered_docs)
        document_prompt = PromptTemplate(
            input_variables=["page_content"], template="{page_content}"
        )
        document_variable_name = "context"
        
        stuff_prompt_override = """Given this text extracts:
        -----
        {context}
        -----
        Please answer the following question:
        {query}"""
        
        
        prompt = PromptTemplate(
            template=stuff_prompt_override, input_variables=["context", "query"]
        )
        
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        chain = StuffDocumentsChain(
            llm_chain=llm_chain,
            document_prompt=document_prompt,
            document_variable_name=document_variable_name,
        )
        response=chain.run(input_documents=reordered_docs, query=query)
        ground_truth = df.iloc[i]['ground truth']  
        result.append({'question': query, 'answer': response, 'ground_truths': ground_truth,'contexts': contexts})
    return result

In [ ]:

result=longcontextreorder(retriever,queries)

### BGE reranker   (run this on gpu)

In [ ]:
class Document:
    def __init__(self, content):
        self.page_content = content
        self.metadata = {}

def read_files(directory_path):
    documents = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_path = Path(root) / file
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                    documents.append(Document(content))
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
    return documents

repo_path = 'Boilerplate'
documents = read_files(repo_path)
docs_texts = [d.page_content for d in documents]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
docs = text_splitter.split_documents(documents)

In [ ]:
from transformers import RagTokenizer, RagTokenForGeneration
from FlagEmbedding import BGEM3FlagModel, FlagReranker
import numpy as np
import faiss
import pandas as pd
from langchain_groq import ChatGroq

tokenizer = RagTokenizer.from_pretrained('facebook/rag-token-nq')
model = RagTokenForGeneration.from_pretrained('facebook/rag-token-nq')
model1 = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False)
reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True)

groq_api_key = "your-key"
client = Groq(api_key=groq_api_key)
llm = ChatGroq(groq_api_key=groq_api_key, model_name='llama3-70b-8192')

In [ ]:
def create_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

def bge_m3_embed(query: str):
    embeddings = model1.encode([query])['dense_vecs'][0]
    return embeddings

def embed_docs(docs):
    contents = [doc.page_content for doc in docs]
    embeddings = np.array([bge_m3_embed(content) for content in contents])
    return embeddings
embeddings = embed_docs(docs)
index = create_faiss_index(embeddings)

In [ ]:
def retrieve_and_rerank_documents(query, index, docs, k=5, top_n=4):
    query_embedding = bge_m3_embed(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, k)
    retrieved_docs = [docs[idx] for idx in indices.flatten()]

    passages = [doc.page_content for doc in retrieved_docs]
    query_passage_pairs = [[query, passage] for passage in passages]
    scores = reranker.compute_score(query_passage_pairs)
    print(scores)
    scored_docs = list(zip(retrieved_docs, scores))
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    top_docs = [doc for doc, score in scored_docs[:top_n]]
    return top_docs

def rag_answer(questions, index, docs, top_n=4):
    result = []
    file_path = 'positive_ground_truths.xlsx'
    df = pd.read_excel(file_path)

    for i, question in enumerate(questions):
        top_docs = retrieve_and_rerank_documents(question, index, docs, top_n=top_n)
        contexts = [doc.page_content for doc in top_docs]  # keep as list

        prompt = f"""Answer the question based only on the following context: {' '.join(contexts)} Question: {question}"""
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model='llama3-70b-8192',
        )
        response = chat_completion.choices[0].message.content

        ground_truth = df.iloc[i]['ground truth']
        result.append({'question': question, 'answer': response, 'ground_truths': ground_truth, 'contexts': contexts})

    return result



result = rag_answer(queries, index, docs)